In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


In [2]:
import json
import requests
import subprocess
import uuid
import time

from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a

from google.adk.agents import Agent
from google.adk.agents import LlmAgent
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.sessions import DatabaseSessionService
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.runners import Runner
from google.genai import types

import warnings

warnings.filterwarnings("ignore")


print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]['base_url']

    try:
        path_parts = baseURL.split('/')
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"
    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """
    display(HTML(styled_html))

    return url_prefix

print("✅ Agent Development Kit Proxy functions defined.")

✅ Agent Development Kit Proxy functions defined.


In [4]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        if(
            (response[i].content.parts)
            and (response[i].contentparts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
            
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code['results'] != "''''":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Schema >>",
                        response_code['result'].replace("tool_code","")
                    )
                else:
                    print("Generated Python Response >>", response_code['result'])

print("Python-Result Display Function Defined")

Python-Result Display Function Defined


In [5]:
root_agent = LlmAgent(
    name = "carelink_bot",
    model = "gemini-2.5-flash-lite",
    description=(
        """You are Carelink Bot, the primary orchestrator in the Carelink AI healthcare support system.
Your role is to manage the end-to-end interaction with the user, convert unstructured questions into structured tasks, and coordinate the work of two specialized subagents.

Core Functions
• Identify the user’s intent and determine whether they need medicine information, lifestyle guidance, clarification, or reassurance.
• Delegate medical information queries to Subagent 1 (Medicine Intelligence Engine).
• Delegate verification, safety review, and lifestyle guidance to Subagent 2 (Health Insight Verifier).
• Request structured outputs from subagents, consolidate the results, and deliver a clear final response.
• Maintain strict boundaries: you do not invent medical facts, diagnose conditions, or override verified data returned by subagents.
• Ensure that every final answer is fact-checked through Subagent 2 before being delivered to the user.
• When the user’s request is out of scope or requires a licensed clinician, redirect them responsibly.

Operating Principles
• Prioritize accuracy, safety, and clarity.
• Use subagents for all medical facts and lifestyle recommendations—never bypass the verification workflow.
• Maintain a user-centric tone: supportive, concise, and actionable.
• Always synthesize information from multiple agents into a single, coherent output.

Interaction Protocol

Interpret the user’s message.

Break it into subtasks if needed.

Route factual lookups to Subagent 1.

Route verification and lifestyle analysis to Subagent 2.

Merge outputs and respond with a final, validated message. !"""
        
    ),
    
    tools=[google_search]
)
print(root_agent)

name='carelink_bot' description='You are Carelink Bot, the primary orchestrator in the Carelink AI healthcare support system.\nYour role is to manage the end-to-end interaction with the user, convert unstructured questions into structured tasks, and coordinate the work of two specialized subagents.\n\nCore Functions\n• Identify the user’s intent and determine whether they need medicine information, lifestyle guidance, clarification, or reassurance.\n• Delegate medical information queries to Subagent 1 (Medicine Intelligence Engine).\n• Delegate verification, safety review, and lifestyle guidance to Subagent 2 (Health Insight Verifier).\n• Request structured outputs from subagents, consolidate the results, and deliver a clear final response.\n• Maintain strict boundaries: you do not invent medical facts, diagnose conditions, or override verified data returned by subagents.\n• Ensure that every final answer is fact-checked through Subagent 2 before being delivered to the user.\n• When th

In [6]:
livebot = InMemoryRunner(agent = root_agent)
print("Live Bot Created !")
response = await livebot.run_debug("Hi , can you tell me what medicine should i take if i have pain in my elbow?")
print(response)

Live Bot Created !

 ### Created new session: debug_session_id

User > Hi , can you tell me what medicine should i take if i have pain in my elbow?
carelink_bot > For pain in your elbow, over-the-counter pain relievers like ibuprofen (Advil, Motrin IB) or naproxen sodium (Aleve) can help reduce pain and inflammation. Acetaminophen (Tylenol) is also an option for pain relief.

Here are some other home care strategies that may help:
*   **Rest:** Avoid activities that aggravate your elbow pain.
*   **Ice:** Apply a cold pack or ice wrapped in a cloth to your elbow for 15-20 minutes, several times a day, especially in the first 48 hours after an injury or flare-up. This helps numb the pain and reduce swelling.
*   **Heat:** After the initial inflammation has subsided, heat therapy can help relax muscles and improve blood flow.
*   **Elevation:** When possible, elevate your arm above your heart to help reduce swelling.
*   **Gentle Exercises:** Once pain begins to decrease, gentle stretchi

In [7]:
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [8]:
search_tool_lm = LlmAgent(
    name = "mie_search",
    model = Gemini(model ="gemini-2.5-flash-lite"),
    description = "Performs medical fact lookup for medicines, dosage, side effects."
)


medical_search_tool = AgentTool(search_tool_lm)

MIE_agent = LlmAgent(
    name = "mie",
    model = Gemini(model="gemini-2.5-flash-lite"),
    description = """
You are MIE — the Medical Intelligence Engine for Carelink.

Your job:
1. Fetch high-quality, factual, up-to-date information about medicines.
2. Use ONLY the provided tools (web_search).
3. Do NOT guess. If data is conflicting or missing, state it clearly.


Rules:
- Use web_search to gather information.
- Extract facts from results.
- Cite all sources (titles, URLs).
- Include evidence snippets (short sentences from each result).
- Provide a final human-readable summary.

""",
    tools=[medical_search_tool]

)

print("MIE Search and MIE model initiated !")

MIE Search and MIE model initiated !


In [9]:
from google.adk.sessions import Session

USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"
session_service = InMemorySessionService()
runner = Runner(agent = MIE_agent, app_name = "default", session_service=session_service)

await run_session(
    runner,
    [
        "What are the side effects of Dolo 650 ?",  # This time, the agent should remember!
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > What are the side effects of Dolo 650 ?


gemini-2.5-flash-lite >  The side effects of Dolo 650, which contains paracetamol (acetaminophen) as its active ingredient, are generally mild and infrequent when taken as directed.

Common side effects include:
*   Nausea
*   Vomiting
*   Stomach pain

Less common or rare side effects may include:
*   Allergic reactions (skin rash, itching, swelling)
*   Blood disorders
*   Liver damage (especially with overdose or chronic heavy alcohol use)
*   Kidney damage (with long-term overuse)

It is important to follow the recommended dosage to avoid serious side effects like liver damage. Alcohol consumption should be limited, and individuals with pre-existing liver or kidney conditions should consult a doctor before use. Be aware of other medications that may also contain paracetamol to prevent accidental overdose. Seek immediate medical attention if severe side effects occur.

Source:
*   Dolo 650 (paracetamol) side effects - general medical knowledge. URL: (Not provided as it's based on ge

In [10]:
from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [11]:
def thiv_web_search(query: str, tool_context = None):
    return {
        "results" : [
            {
                "title": f"Search results for: {query}",
                "url": "https://example.org/dummy",
                "snippet": f"Dummy snippet confirming: {query}"
            }
        ]
    }
    
    
web_search_tool = LlmAgent(
    name = "thiv_search_tool",
    model = Gemini(model ="gemini-2.5-flash-lite"),
    description ="Fetching dietary, Water intake and Lifestyle Information upon recieving call from MIE"
)

THIV_search = AgentTool(web_search_tool)
THIV_description ="""\n You are THIV — The Health Insight Verifier.

Your input from the caller (Core Agent or MIE) is a single JSON object (string form) containing
the MIE output for a medication lookup. Your responsibilities:

1. Parse the incoming MIE JSON. Confirm required fields are present:
   - request_id
   - drug_normalized
   - indications
   - typical_dosing
   - administration
   - common_side_effects
   - major_interactions
   - contraindications
   - confidence
   - sources
   - evidence_snippets
   - human_summary

2. Verify claims:
   - For each major claim (dosing, interactions, contraindications), attempt to verify via available sources.
   - Use the provided `thiv_web_search` tool when necessary to fetch corroborating sources and evidence snippets.
   - If source lists conflict, record both and mark `disagreement:true`.

3. Score/adjust confidence:
   - If MIE confidence >= 0.8 and you find corroborating sources, keep or increase confidence.
   - If you find contradictions, reduce confidence and set `requires_escalation` appropriately.

4. Enrich with lifestyle guidance (non-clinical, safety-first):
   - Provide water intake hints, sleep window recommendations, and simple diet notes where relevant (e.g., "take with food", "avoid grapefruit").
   - Never give a diagnosis. Give behavioural, conservative, and non-prescriptive suggestions.
   - If a claim requires clinician input (e.g., dosing adjustments for renal impairment), mark `escalation:true` and include the rationale.

5. Urgent flags:
   - If the MIE notes urgent red flags (e.g., "shortness of breath", "severe rash", "jaundice"), set `urgent_flag:true` and provide immediate instructions (call emergency services, seek immediate care).
6. Always include `original_mie` so the Core Agent can audit the chain."""

THIV = LlmAgent(
    name ="thiv",
    model = Gemini(model ="gemini-2.5-flash-lite"),
    description = THIV_description,
    tools = [THIV_search]
)

print("THIV Search and Agent Initiated ")

THIV Search and Agent Initiated 


In [12]:
from google.adk.sessions import Session

USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"
session_service = InMemorySessionService()
runner = Runner(agent = THIV, app_name = "default", session_service=session_service)

await run_session(
    runner,
    [
        "What are the Dietary constraints I should keep in mind while following a veg diet during diarhea?",  # This time, the agent should remember!
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > What are the Dietary constraints I should keep in mind while following a veg diet during diarhea?


gemini-2.5-flash-lite >  While experiencing diarrhea on a vegetarian diet, focus on easily digestible, low-fiber, and bland foods.

**Foods to include:**
*   Ripe bananas
*   White rice (plain, boiled, or steamed)
*   Unsweetened applesauce
*   Plain white toast
*   Well-cooked and peeled vegetables like carrots and potatoes
*   Plain oatmeal or cream of wheat
*   Plain, unsweetened yogurt or curd with live cultures (if dairy is tolerated)
*   Clear vegetable broths
*   Boiled or steamed tofu

**Crucially, stay hydrated** with water, oral rehydration solutions, clear broths, diluted fruit juices (without pulp), and herbal teas.

**Foods to avoid:**
*   High-fiber foods (whole grains, raw fruits/vegetables, legumes, nuts, seeds)
*   Spicy, fried, or greasy foods
*   Most dairy products (except plain yogurt/curd if tolerated)
*   Sugary foods and artificial sweeteners
*   Caffeinated beverages and alcohol
*   Processed foods

It's advisable to eat small, frequent meals and chew thoroughl